In [30]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
import keras.models
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Engineering Features

In [14]:
# select features 'mel', 'mfcc' or 'freq_power'
def selectFeatures(feature='mel'):
    feat = os.path.join("bird_data","features")
    train_folder = os.path.join(feat,f'{feature}_features', 'train')
    test_folder = os.path.join(feat,f'{feature}_features', 'test')
    val_folder = os.path.join(feat,f'{feature}_features', 'val')
    return train_folder, test_folder, val_folder
    

In [15]:
train_folder, test_folder, val_folder = selectFeatures('mel')

# Neural Network

In [18]:
# Batch and target size
BATCH_SIZE = 16
TARGET_SIZE = (720,288)


In [3]:
def selectModel(BATCH_SIZE=32, TARGET_SIZE=(728,288)):
    model = keras.models.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(720, 288, 3)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model.summary()
    

In [6]:
selectModel()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 718, 286, 32)      896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 716, 284, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 358, 142, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 358, 142, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3253504)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                208224320 
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)               

In [34]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping]

In [10]:
def dataGenerator(feed='TRAIN'):
    if feed=='TRAIN':
        return ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip = False)
    elif feed=='TEST':
        return ImageDataGenerator(preprocessing_function=preprocess_input)
    else:
        print('Wrong argument. Enter "TRAIN"  or "TEST"')

In [22]:
train_dg=dataGenerator()
test_dg=dataGenerator('TEST')

In [23]:
def datagenFlow(DATAGEN, FOLDER, TARGET_SIZE, BATCH_SIZE, SHUFFLE=True):
    return DATAGEN.flow_from_directory(FOLDER, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical', shuffle=SHUFFLE)

In [ ]:
train_set = datagenFlow(train_dg, train_folder, TARGET_SIZE, BATCH_SIZE)

In [ ]:
test_set= datagenFlow(test_dg, test_folder, TARGET_SIZE, BATCH_SIZE, False)

In [ ]:
val_set= datagenFlow(test_dg, val_folder, TARGET_SIZE, BATCH_SIZE, False)

In [27]:
def classWeights(tset):
    class_weights = class_weight.compute_class_weight('balanced', np.unique(tset.classes), tset.classes)
    weights = {label:w for label, w in zip(np.unique(training_set.classes), class_weights)}
    return weights
    

In [ ]:
weights = classWeights(train_set)

In [ ]:
model.fit(train_set, epochs=10, class_weight=weights,  validation_data = test_set, verbose=True, callbacks=callbacks, shuffle=True)

In [28]:
def Score(tset):
    score = model.evaluate(tset, verbose=0)
    return score[1]

In [ ]:
train_score = Score(train_set)
test_score = Score(test_set)
val_score = Score(val_set)[1]

In [ ]:
print(train_score, test_score, val_score)

### run in Google Colab